In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import xml

!conda install -c conda-forge geopy
!conda install -c conda-forge folium=0.5.0 

from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [2]:
url = "https://www.mapsofindia.com/pincode/india/tamil-nadu/chennai/"
table = pd.read_html(url, header=0,keep_default_na=False)
table[5]
chennai = table[5][['Location', 'Pincode']]

chennai.head()

,Location,Pincode
0,Abiramapuram,600018
1,Adyar,600020
2,Agaram,600082
3,Aminjikarai,600029
4,Anna Nagar,600040


In [3]:
# The code was removed by Watson Studio for sharing.

,Pincode,Location,City,Latitude,Longitude
0,600001,Chennai G.P.O.,Chennai,13.0933,80.2842
1,600001,MPT AO,Chennai,13.0933,80.2842
2,600001,Flower Bazaar,Chennai,13.0933,80.2842
3,600001,Sowcarpet,Chennai,13.0897,80.2803
4,600001,Mannady (Chennai),Chennai,13.0969,80.2881


In [4]:
tdf1 = pd.merge(chennai, tdf, on=['Location','Pincode'])
tdf1.head()

,Location,Pincode,City,Latitude,Longitude
0,Agaram,600082,Chennai,13.1100,80.2333
1,Aminjikarai,600029,Chennai,13.0739,80.2244
2,Anna Road,600002,Chennai,13.0744,80.2714
3,Arumbakkam,600106,Chennai,13.0734,80.2069
4,Aynavaram,600023,Chennai,13.0622,80.2409


In [5]:
address = 'Chennai,India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Chennai,India are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Chennai,India are 13.0801721, 80.2838331.


In [6]:
# create map of Chennai using latitude and longitude values
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, nei in zip(tdf1['Latitude'], tdf1['Longitude'], tdf1['Location']):
   
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chennai)  
    
map_chennai

In [7]:
CLIENT_ID = 'VYD3QINZN4USUENKLENHRB43QIW5JUPP33WO0TUG3SSZZZCE' # your Foursquare ID
CLIENT_SECRET = 'Z2EEKXR2FPHICRMI41JWK10LYVS30C1RDWWV32XB2ON3TJ1H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version# The code was removed by Watson Studio for sharing.

In [8]:
radius = 1000 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
   longitude, 
    radius, 
   LIMIT)


In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c06c5571ed2194a7707e388'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cb9863b90c9a143bac487d6-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d181941735',
         'name': 'Museum',
         'pluralName': 'Museums',
         'primary': True,
         'shortName': 'Museum'}],
       'id': '4cb9863b90c9a143bac487d6',
       'location': {'cc': 'IN',
        'country': 'India',
        'distance': 424,
        'formattedAddress': ['India'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 13.080618432998454,
          'lng': 80.2877188704723}],
        'lat': 13.080618432998454,
        'lng': 80.2877188704723},
       'name': 'F

In [10]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng', 'venue.location.state',
       'venue.name', 'venue.photos.count', 'venue.photos.groups'],
      dtype='object')

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Fort Museum,"[{'primary': True, 'shortName': 'Museum', 'plu...",13.080618,80.287719
1,Higginbothams,"[{'primary': True, 'shortName': 'Bookstore', '...",13.083190,80.275168
2,Parrys,"[{'primary': True, 'shortName': 'Market', 'plu...",13.088422,80.283283
3,Evening bazaar,"[{'primary': True, 'shortName': 'Convenience S...",13.085633,80.282265
4,Island Grounds,"[{'primary': True, 'shortName': 'Campground', ...",13.073727,80.283485
5,Chennai Port,"[{'primary': True, 'shortName': 'Harbor / Mari...",13.085479,80.290026
6,Chennai Park Town MRTS,"[{'primary': True, 'shortName': 'Train Station...",13.079579,80.275484
7,Central Bus Stand,"[{'primary': True, 'shortName': 'Bus Station',...",13.081912,80.275315
8,New Prakash Bhavan,"[{'primary': True, 'shortName': 'Indian', 'plu...",13.087493,80.279310
9,City Tower Hotel Chennai,"[{'primary': True, 'shortName': 'Hotel', 'plur...",13.081503,80.275015


In [13]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Fort Museum,Museum,13.080618,80.287719
1,Higginbothams,Bookstore,13.083190,80.275168
2,Parrys,Market,13.088422,80.283283
3,Evening bazaar,Convenience Store,13.085633,80.282265
4,Island Grounds,Campground,13.073727,80.283485
5,Chennai Port,Harbor / Marina,13.085479,80.290026
6,Chennai Park Town MRTS,Train Station,13.079579,80.275484
7,Central Bus Stand,Bus Station,13.081912,80.275315
8,New Prakash Bhavan,Indian Restaurant,13.087493,80.279310
9,City Tower Hotel Chennai,Hotel,13.081503,80.275015


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
chennai_venues = getNearbyVenues(names=tdf1['Location'],
                                   latitudes=tdf1['Latitude'],
                                   longitudes=tdf1['Longitude']
                                  )

Agaram
Aminjikarai
Anna Road
Arumbakkam
Aynavaram
Besantnagar
Chepauk
Chetput
Chintadripet
Choolaimedu
Edapalayam
Egmore
Ekkaduthangal
Erukkancheri
Ethiraj Salai
Flowers Road
Greams Road
Guindy North
Icf Colony
Jafferkhanpet
Kasturibai Nagar
Kilpauk
Kodambakkam
Kodungaiyur
Kotturpuram
Koyambedu
Lloyds Estate
Loyola College
Madras University
Mandaveli
Mylapore
Nandanam
Nungambakkam
Park Town
Perambur
Perambur Barracks
Perambur North
Pudupakkam
Puliyanthope
Raja Annamalaipuram
Rayapuram
Ripon Buildings
Royapettah
Rv Nagar
Saligramam
Sembiam
Shastri Bhavan
Shenoy Nagar
Teynampet
Teynampet West
Tidel Park
Tiruvallikkeni
Tiruvanmiyur
Tondiarpet
Tondiarpet Bazaar
Tondiarpet West
Triplicane South
Vadapalani
Velacheri
Venkatesapuram
Vepery
Virugambakkam
Vyasarpadi
Washermanpet
Washermanpet East
West Mambalam


In [16]:
print(chennai_venues.shape)

(907, 7)


In [17]:
chennai_venues.groupby('Location').count()

,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Location,,,,,,
Agaram,5,5,5,5,5,5
Aminjikarai,15,15,15,15,15,15
Anna Road,4,4,4,4,4,4
Arumbakkam,7,7,7,7,7,7
Aynavaram,33,33,33,33,33,33
Chepauk,13,13,13,13,13,13
Chetput,31,31,31,31,31,31
Chintadripet,4,4,4,4,4,4
Edapalayam,20,20,20,20,20,20


In [18]:
print('There are {} uniques categories.'.format(len(chennai_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [19]:
# one hot encoding
chennai_onehot = pd.get_dummies(chennai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chennai_onehot['Location'] = chennai_venues['Location'] 

# move neighborhood column to the first column
fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]
chennai_onehot.head()

,Location,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bookstore,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Library,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Flower Shop,Food,Frozen Yogurt Shop,Furniture / Home Store,Garden,General Travel,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Light Rail Station,Lighthouse,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Nightclub,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Record Shop,Resort,Restaurant,Russian Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Sports Club,Tennis Stadium,Thai Restaurant,Theater,Train,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,Agaram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agaram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Agaram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Agaram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Agaram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
chennai_onehot.shape

(907, 108)

In [21]:
chennai_grouped = chennai_onehot.groupby('Location').mean().reset_index()
chennai_grouped.head()

,Location,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bookstore,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Library,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Flower Shop,Food,Frozen Yogurt Shop,Furniture / Home Store,Garden,General Travel,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Light Rail Station,Lighthouse,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Nightclub,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Record Shop,Resort,Restaurant,Russian Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Sports Club,Tennis Stadium,Thai Restaurant,Theater,Train,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,Agaram,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0
1,Aminjikarai,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.200000,0.00,0.0,0.066667,0.0,0.066667,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.00,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0
2,Anna Road,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.000000,0.0,0.0
3,Arumbakkam,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.285714,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.142857,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.285714,0.0,0.0
4,Aynavaram,0.0,0.0,0.0,0.030303,0.0,0.030303,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.030303,0.0,0.060606,0.060606,0.030303,0.030303,0.0,0.0,0.0,0.06060

In [22]:
chennai_grouped.shape

(63, 108)

In [23]:
num_top_venues = 8
for loc in chennai_grouped['Location']:
    print("----"+loc+"----")
    temp =chennai_grouped[chennai_grouped['Location'] == loc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agaram----
                           venue  freq
0                      Multiplex   0.8
1                    Pizza Place   0.2
2             African Restaurant   0.0
3                     Playground   0.0
4                       Pharmacy   0.0
5          Performing Arts Venue   0.0
6                           Park   0.0
7  Paper / Office Supplies Store   0.0


----Aminjikarai----
                    venue  freq
0    Fast Food Restaurant  0.20
1             Pizza Place  0.13
2       Electronics Store  0.07
3          Clothing Store  0.07
4           Shopping Mall  0.07
5               Multiplex  0.07
6  Furniture / Home Store  0.07
7               Bookstore  0.07


----Anna Road----
                            venue  freq
0                   Movie Theater  0.25
1                           Train  0.25
2                     Flea Market  0.25
3                  General Travel  0.25
4  Multicuisine Indian Restaurant  0.00
5                        Pharmacy  0.00
6           Performing A

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
location_venues_sorted = pd.DataFrame(columns=columns)
location_venues_sorted['Location'] = chennai_grouped['Location']

for ind in np.arange(chennai_grouped.shape[0]):
    location_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chennai_grouped.iloc[ind, :], num_top_venues)

location_venues_sorted

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agaram,Multiplex,Pizza Place,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
1,Aminjikarai,Fast Food Restaurant,Pizza Place,Shopping Mall,Food,Indian Restaurant,Electronics Store,Multiplex,Bookstore,Clothing Store,Metro Station
2,Anna Road,Train,Flea Market,General Travel,Movie Theater,Yoga Studio,Frozen Yogurt Shop,Department Store,Dessert Shop,Diner,Eastern European Restaurant
3,Arumbakkam,Vegetarian / Vegan Restaurant,Bus Station,Bus Line,Hotel,Fast Food Restaurant,Yoga Studio,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store
4,Aynavaram,Indian Restaurant,Convenience Store,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Café,Gourmet Shop,Multicuisine Indian Restaurant,Coffee Shop
5,Chepauk,Indian Restaurant,Sports Club,Platform,Music Store,Flea Market,Light Rail Station,Beach,Cricket Ground,Vegetarian / Vegan Restaurant,Train Station
6,Chetput,Indian Restaurant,Café,Hotel,Department Store,Bakery,Coffee Shop,Chinese Restaurant,Juice Bar,Gourmet Shop,Fast Food Restaurant
7,Chintadripet,Train,Flea Market,General Travel,Movie Theater,Yoga Studio,Frozen Yogurt Shop,Department Store,Dessert Shop,Diner,Eastern European Restaurant
8,Edapalayam,Indian Restaurant,Train Station,Bookstore,Bus Station,Hotel,Furniture / Home Store,Men's Store,Nightclub,Outlet Store,Platform
9,Egmore,Indian Restaurant,Café,Bowling Alley,Seafood Restaurant,Hotel,Shopping Mall,Museum,Fast Food Restaurant,Mexican Restaurant,Sandwich Place


In [26]:
tdf2 = pd.merge(tdf1, location_venues_sorted, on=['Location'], how='outer')


tdf3 = tdf2 [['Location','Pincode','Latitude','Longitude']]

tdf3

,Location,Pincode,Latitude,Longitude
0,Agaram,600082,13.1100,80.2333
1,Aminjikarai,600029,13.0739,80.2244
2,Anna Road,600002,13.0744,80.2714
3,Arumbakkam,600106,13.0734,80.2069
4,Aynavaram,600023,13.0622,80.2409
5,Besantnagar,600090,13.8144,80.2791
6,Chepauk,600005,13.0594,80.2789
7,Chetput,600031,13.0700,80.2408
8,Chintadripet,600002,13.0744,80.2714
9,Choolaimedu,600094,13.8144,80.2791


In [27]:
# set number of clusters
# Using k-means to cluster the neighborhood into 8 clusters.
kclusters = 4

chennai_grouped_clustering = chennai_grouped.drop('Location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chennai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 1, 2, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 3, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 2, 3, 2, 0, 3, 1, 1, 0, 0, 1, 0, 2, 2, 0,
       0, 2, 1, 0, 0, 1, 1, 1, 0, 1, 3, 0, 1, 1, 1, 1, 1], dtype=int32)

In [28]:
clusterfr = pd.DataFrame()

# add clustering labels
clusterfr['Cluster Labels'] = kmeans.labels_
tdf4 = tdf3.join(clusterfr, how='outer')
tdf4.head()

,Location,Pincode,Latitude,Longitude,Cluster Labels
0,Agaram,600082,13.1100,80.2333,2.0
1,Aminjikarai,600029,13.0739,80.2244,1.0
2,Anna Road,600002,13.0744,80.2714,2.0
3,Arumbakkam,600106,13.0734,80.2069,1.0
4,Aynavaram,600023,13.0622,80.2409,0.0


In [29]:
chennai_merged = pd.merge(tdf4, location_venues_sorted, on='Location')
chennai_merged = chennai_merged[np.isfinite(chennai_merged['Cluster Labels'])]
chennai_merged_clean = chennai_merged.copy()
chennai_merged_clean ['Cluster Labels'] = chennai_merged_clean['Cluster Labels'].astype(int)
chennai_merged_clean.head()

,Location,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agaram,600082,13.1100,80.2333,2,Multiplex,Pizza Place,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
1,Aminjikarai,600029,13.0739,80.2244,1,Fast Food Restaurant,Pizza Place,Shopping Mall,Food,Indian Restaurant,Electronics Store,Multiplex,Bookstore,Clothing Store,Metro Station
2,Anna Road,600002,13.0744,80.2714,2,Train,Flea Market,General Travel,Movie Theater,Yoga Studio,Frozen Yogurt Shop,Department Store,Dessert Shop,Diner,Eastern European Restaurant
3,Arumbakkam,600106,13.0734,80.2069,1,Vegetarian / Vegan Restaurant,Bus Station,Bus Line,Hotel,Fast Food Restaurant,Yoga Studio,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store
4,Aynavaram,600023,13.0622,80.2409,0,Indian Restaurant,Convenience Store,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Café,Gourmet Shop,Multicuisine Indian Restaurant,Coffee Shop


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_merged_clean['Latitude'], chennai_merged_clean['Longitude'], chennai_merged_clean['Location'], chennai_merged_clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
cdf1=chennai_merged.loc[chennai_merged['Cluster Labels'] == 0,chennai_merged.columns[[0] + list(range(5, chennai_merged.shape[1]))]]
cdf1.loc[cdf1['1st Most Common Venue'] == 'Multiplex']

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Sembiam,Multiplex,Shopping Mall,Bus Station,Pizza Place,Indian Restaurant,Bakery,Restaurant,Dessert Shop,Diner,Eastern European Restaurant
55,Vadapalani,Multiplex,Coffee Shop,Clothing Store,Indian Restaurant,Café,Asian Restaurant,Hotel,Pizza Place,Fast Food Restaurant,Restaurant


In [32]:
cdf2=chennai_merged.loc[chennai_merged['Cluster Labels'] == 1,chennai_merged.columns[[0] + list(range(5, chennai_merged.shape[1]))]]
cdf2.loc[cdf2['1st Most Common Venue'] == 'Multiplex']

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [33]:
cdf3=chennai_merged.loc[chennai_merged['Cluster Labels'] == 2,chennai_merged.columns[[0] + list(range(5, chennai_merged.shape[1]))]]
cdf3.loc[cdf3['1st Most Common Venue'] == 'Multiplex']

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agaram,Multiplex,Pizza Place,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
32,Perambur,Multiplex,Shopping Mall,Bus Station,Pizza Place,Indian Restaurant,Bakery,Restaurant,Dessert Shop,Diner,Eastern European Restaurant


In [34]:
cdf4=chennai_merged.loc[chennai_merged['Cluster Labels'] == 3,chennai_merged.columns[[0] + list(range(5, chennai_merged.shape[1]))]]
cdf4.loc[cdf4['1st Most Common Venue'] == 'Multiplex']

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Perambur North,Multiplex,Shopping Mall,Bus Station,Pizza Place,Indian Restaurant,Bakery,Restaurant,Dessert Shop,Diner,Eastern European Restaurant
